**Upload File**

Upload a JSON file exported from GTM under Admin > Export Container

In [ ]:
from google.colab import files
uploaded = files.upload()

**Parse the Tags**

Drill down into the tag section of the JSON export. For each tag, copy the tag settings that are common to all tags using the same name as in the export and then copy the parameters in the export that contain the unique configurations of each tag type adding a parameter_ prefix to identify parameters downstream.

Save the tag data to a dictionary and list the first five entries.

In [ ]:
%%capture
!pip install -U 'fsspec[s3]'

import json
import pandas as pd

uploaded_json = list(uploaded.values())[0]

container = json.loads(uploaded_json.decode('utf-8'))
# drill down to the tags section of the container
# if you wanted to list something other than tags, this is where you would start making changes
tags = container['containerVersion']['tag'] 

rows = []

for tag in tags:
  row = {}
  if (tag['type'] == 'gaawe' ):
    row['tagId'] = str(tag['tagId'])
    row['name'] = str(tag['name'])
    if "parameter" in tag:
      for p in tag['parameter']:
        if p['key'] == "eventName":
          row['eventName'] = p['value']
        if p['key'] == 'userProperties':
          for li in p['list']:
            key = "default_key"
            value = "default_value"  
            for item in li['map']:        
              if item['key'] == 'name':
                key = 'userProp_' + item['value']
              if item['key'] == 'value':
                value = str(item['value'])
              row[key] = value
        if p['key'] == 'eventSettingsTable':
          for li in p['list']:
            key = "default_key"
            value = "n/a" 
            for item in li['map']:         
              if item['type'] == 'TEMPLATE' and item['key'] == 'parameter':
                key = 'eventParam_' + item['value']
              if item['type'] == 'TEMPLATE' and item['key'] == 'parameterValue':
                value = item['value']
              if key != "default_key":
                row[key] = value
    rows.append(row)
rows[:5]





**Convert to DataFrame**

Extract the container ID to use as the name of the Google Sheet and then convert the dictionary to a DataFrame. 

The resulting DataFrame will have a lot of null values in the parameter_-prefixed columns because the parameters are unique to each tag type.



In [ ]:
import re
container_id = re.findall( "GTM-[A-Z0-9]+" , str(uploaded.keys())) 

data = pd.DataFrame(rows)
data.head()

**Authenticate**

This will open a new window where you need to log in to Google Accounts, give this Colab permission to access Drive, copy the key that gets generated after giving permission, and finally paste that key back here and complete the authentication.

*2022-04-19: Updated authentication method*

In [ ]:
# authenticate
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

**Save to Google Sheets**

The will save as a GSheet in your Google Drive with the title of "GTM Tags for GTM-CONTAINERID." It will take a few minutes for the data to fully populate in the sheet so be patient.

In [ ]:
# create, and save df
from gspread_dataframe import set_with_dataframe
title = 'GTM GA4 Tags for ' + container_id[0]
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet, data) 
# include_index=False, include_column_header=True, resize=False